# Step 1 - Training and testing HerdNet

# Installations

In [ ]:
import os
import logging
from IPython.display import clear_output
from google.colab import drive

class GoogleDriveConnector:
    """
    GoogleDriveConnector
    --------------------
    Clase para montar Google Drive en un entorno de Google Colab.
    
    Propósito
    ---------
    Permite establecer una conexión con Google Drive desde Colab de forma
    controlada, con validación de parámetros, registro de eventos (logging)
    y limpieza visual del entorno tras la conexión.
    
    Parámetros
    ----------
    mount_path : str
        Ruta local donde se montará Google Drive (por defecto '/content/drive').
    verbose : bool, opcional
        Si es True, se habilita el registro mediante logging. Si es False,
        no se muestra ninguna salida ni registro (por defecto True).
    
    Métodos
    -------
    mount_drive():
        Monta Google Drive y registra el proceso.
    """
    
    def __init__(self, mount_path: str = "/content/drive", verbose: bool = True):
        """
        Inicializa la clase configurando el logger y validando la ruta de montaje.
        
        Parámetros
        ----------
        mount_path : str
            Ruta donde se montará Google Drive.
        verbose : bool
            Determina si se habilita el registro de eventos.
        
        Excepciones
        -----------
        ValueError:
            Si 'mount_path' no es una cadena válida o está vacía.
        """
        # Validar tipo y contenido del parámetro
        if not isinstance(mount_path, str) or not mount_path.strip():
            raise ValueError("El parámetro 'mount_path' debe ser una cadena no vacía.")
        
        # Asignar atributos de instancia
        self.mount_path = mount_path
        self.verbose = verbose
        
        # Configurar logger si está habilitado
        self.logger = logging.getLogger("GoogleDriveConnector")
        if self.verbose:
            self.logger.setLevel(logging.INFO)
            if not self.logger.handlers:
                handler = logging.StreamHandler()
                formatter = logging.Formatter(
                    "[%(asctime)s] [%(levelname)s] %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                )
                handler.setFormatter(formatter)
                self.logger.addHandler(handler)
            self.logger.propagate = False
        else:
            self.logger.disabled = True
    
    def mount_drive(self) -> None:
        """
        Monta Google Drive en el entorno actual de Google Colab.
        
        Retorno
        -------
        None
        
        Excepciones
        -----------
        RuntimeError:
            Si ocurre un error durante el proceso de montaje.
        """
        # Registrar inicio del proceso de conexión
        if self.verbose:
            self.logger.info("Iniciando conexión con Google Drive...")
        
        # Nueva verificación: evitar conflicto si la ruta ya tiene archivos
        if os.path.exists(self.mount_path) and os.listdir(self.mount_path):
            if self.verbose:
                self.logger.warning(
                    f"La ruta '{self.mount_path}' ya contiene archivos. "
                    "Se asume que Google Drive ya está montado."
                )
            return
        
        try:
            # Montar Google Drive
            drive.mount(self.mount_path)
            
            # Limpiar salida visual del entorno
            clear_output(wait=True)
            
            # Confirmar éxito del proceso
            if self.verbose:
                self.logger.info("Google Drive montado correctamente.")
            else:
                print("Google Drive montado correctamente.")
        
        except Exception as e:
            # Manejar y registrar errores
            if self.verbose:
                self.logger.error(f"Error al montar Google Drive: {e}", exc_info=True)
            raise RuntimeError(f"No se pudo montar Google Drive: {e}") from e

In [ ]:
# Conectar a Google Drive
CONNECTOR = GoogleDriveConnector(
    mount_path="/content/drive",
    verbose=True,
)
CONNECTOR.mount_drive()

In [ ]:
import os
import sys
import logging
import subprocess
from pathlib import Path
from IPython.display import clear_output

class ConfiguradorHerdNet:
    """
    Clase para configurar automáticamente el entorno de HerdNet,
    compatible tanto con Google Colab como con entornos locales.
    
    Propósito
    ---------
    Reproduce la instalación de HerdNet con detección de entorno,
    manejo de errores y registro de eventos mediante logging.
    
    Parámetros
    ----------
    base_dir : str, opcional
        Ruta donde se clonará el repositorio HerdNet.
        Por defecto: '/content/HerdNet' si se ejecuta en Colab,
        o './HerdNet' si se ejecuta localmente.
    verbose : bool, opcional
        Si es True, muestra información detallada del proceso.
    """
    
    def __init__(self, base_dir: str = None, verbose: bool = True):
        """Inicializa el configurador y detecta el entorno de ejecución."""
        
        # Detectar si se ejecuta en Colab
        self.is_colab = "google.colab" in sys.modules
        
        # Definir ruta base según entorno
        self.base_dir = Path(base_dir or ("/content/HerdNet" if self.is_colab else "./HerdNet"))
        
        # Configurar logging estándar
        logging.basicConfig(
            level=logging.INFO,
            format="[%(asctime)s] [%(levelname)s] %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
            force=True
        )
        self.logger = logging.getLogger("ConfiguradorHerdNet")
        
        # Control de verbosidad
        self.verbose = verbose
    
    # ----------------------------------------------------------------------
    
    def _run_command(self, command: str):
        """
        Ejecuta un comando del sistema de forma segura.
        
        Parámetros
        ----------
        command : str
            Comando del sistema a ejecutar.
        
        Excepciones
        -----------
        subprocess.CalledProcessError
            Si el comando devuelve un error de ejecución.
        """
        try:
            result = subprocess.run(
                command,
                shell=True,
                check=True,
                text=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            if self.verbose and result.stdout.strip():
                self.logger.info(result.stdout.strip())
        except subprocess.CalledProcessError as e:
            self.logger.error(f"Error ejecutando comando: {command}")
            if e.stderr.strip():
                self.logger.error(e.stderr.strip())
            raise
    
    # ----------------------------------------------------------------------
    
    def verificar_gpu(self):
        """Verifica la disponibilidad de una GPU mediante nvidia-smi."""
        self.logger.info("Verificando GPU disponible...")
        try:
            self._run_command("nvidia-smi")
        except Exception:
            self.logger.warning("No se pudo verificar GPU. Puede que no exista o no esté disponible.")
    
    # ----------------------------------------------------------------------
    
    def instalar_dependencias(self):
        """
        Instala las dependencias necesarias según el entorno detectado.
        
        En Colab se instalan versiones recientes compatibles.
        En local se instalan las versiones exactas del paper HerdNet.
        """
        if self.is_colab:
            self.logger.info("Entorno detectado: Google Colab")
            self.logger.info("Instalando dependencias compatibles con Colab...")
            deps = (
                "albumentations fiftyone hydra-core opencv-python pandas pillow "
                "scikit-image scikit-learn scipy wandb"
            )
        else:
            self.logger.info("Entorno detectado: Local")
            self.logger.info("Instalando dependencias exactas del paper HerdNet...")
            deps = (
                "albumentations==1.0.3 fiftyone==0.14.3 hydra-core==1.1.0 "
                "opencv-python==4.5.1.48 pandas==1.2.3 pillow==8.2.0 "
                "scikit-image==0.18.1 scikit-learn==1.0.2 scipy==1.6.2 wandb==0.10.33"
            )
        
        cmd = f"{sys.executable} -m pip install {deps} -q"
        try:
            self._run_command(cmd)
        except Exception as e:
            self.logger.warning(f"Fallo parcial en instalación de dependencias: {e}")
            self.logger.warning("Continuando con el proceso...")
    
    # ----------------------------------------------------------------------
    
    def clonar_repo(self):
        """Clona e instala el repositorio HerdNet desde GitHub."""
        self.logger.info("Clonando repositorio HerdNet original...")
        if self.base_dir.exists():
            self.logger.info("El repositorio ya existe. Se omite la clonación.")
        else:
            self._run_command(f"git clone https://github.com/Alexandre-Delplanque/HerdNet {self.base_dir}")
        
        self._run_command(f"cd {self.base_dir} && {sys.executable} setup.py install -q")
        sys.path.append(str(self.base_dir))
    
    # ----------------------------------------------------------------------
    
    def limpiar_salida(self):
        """Limpia la salida si se ejecuta en Google Colab."""
        if self.is_colab:
            clear_output(wait=True)
    
    # ----------------------------------------------------------------------
    
    def configurar(self):
        """
        Ejecuta el flujo completo de configuración del entorno HerdNet.
        
        Incluye:
        - Verificación de GPU
        - Instalación de dependencias
        - Clonación del repositorio
        - Instalación local
        - Limpieza de salida (solo en Colab)
        """
        self.logger.info("Iniciando configuración del entorno HerdNet...")
        try:
            self.verificar_gpu()
            self.instalar_dependencias()
            self.clonar_repo()
            self.limpiar_salida()
            self.logger.info("Instalación completada correctamente y entorno listo.")
            print("Instalación completada correctamente y entorno listo.")
        except Exception as e:
            self.logger.error(f"Error durante la configuración: {e}")
            raise

In [ ]:
# Instsalación de librerías y clonación de animaloc
configurador = ConfiguradorHerdNet(verbose=True)
configurador.configurar()

In [ ]:
from animaloc.utils.seed import set_seed

class FijadorSemilla:
    """
    FijadorSemilla
    --------------
    Establece una semilla aleatoria determinística para los módulos de Python,
    NumPy y PyTorch, garantizando la reproducibilidad de los experimentos.
    
    Propósito
    ---------
    Proporcionar una interfaz unificada para fijar el valor de la semilla que
    utilizan los generadores aleatorios del entorno.
    
    Parámetros
    ----------
    seed : int
        Valor entero que se usará como semilla global.
    
    Ejemplo
    -------
    >>> fijador = FijadorSemilla(seed=9292)
    >>> fijador.aplicar()
    """
    
    def __init__(self, seed: int = 9292):
        # Guarda el valor de la semilla
        self.seed = seed
    
    def aplicar(self):
        """
        Aplica la semilla determinística en Python, NumPy y PyTorch.
        
        Retorno
        -------
        None
            El método no devuelve ningún valor; solo fija la semilla global.
        """
        # Fija la semilla de manera reproducible
        set_seed(self.seed)
        
        # Imprime confirmación
        print(f"Semilla aleatoria fijada en {self.seed}")

In [ ]:
# Fijar semilla global
fijador = FijadorSemilla(seed=9292)
fijador.aplicar()

# Create datasets

In [ ]:
# Download the data of Delplanque et al. (2021)
!gdown 1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st -O /content/data.zip
!unzip -oq /content/data.zip -d /content

# URL: https://drive.google.com/uc?id=1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st

# Create validation patches using the patcher tool (for demo)
from animaloc.utils.useful_funcs import mkdir

mkdir('/content/data/val_patches')

!python /content/HerdNet/tools/patcher.py /content/data/val 512 512 0 /content/data/val_patches -csv /content/data/val.csv -min 0.0 -all False

#!cp -r /content/data/val_patches /content/drive/MyDrive/HerdNet/

In [ ]:
import albumentations as A
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

class HerdNetDatasetsBuilder:
    """
    Clase para construir los datasets de entrenamiento, validación y prueba
    utilizados por el modelo HerdNet.
    
    Esta clase encapsula la configuración de transformaciones de Albumentations
    y las transformaciones finales requeridas por el modelo.
    
    Parámetros
    ----------
    train_csv : str
        Ruta al archivo CSV de entrenamiento.
    train_root : str
        Carpeta raíz donde se encuentran las imágenes de entrenamiento.
    val_csv : str
        Ruta al archivo CSV de validación.
    val_root : str
        Carpeta raíz de las imágenes de validación.
    test_csv : str
        Ruta al archivo CSV de prueba.
    test_root : str
        Carpeta raíz de las imágenes de prueba.
    patch_size : int, opcional
        Tamaño del parche utilizado. Por defecto es 512.
    num_classes : int, opcional
        Número total de clases (incluye fondo). Por defecto es 7.
    down_ratio : int, opcional
        Factor de reducción espacial. Por defecto es 2.
    
    Retorna
    -------
    tuple
        Una tupla con tres objetos:
        (train_dataset, val_dataset, test_dataset)
    """
    
    def __init__(
        self,
        train_csv: str,
        train_root: str,
        val_csv: str,
        val_root: str,
        test_csv: str,
        test_root: str,
        patch_size: int = 512,
        num_classes: int = 7,
        down_ratio: int = 2,
    ):
        # Se guardan las rutas y parámetros como atributos
        self.train_csv = train_csv
        self.train_root = train_root
        self.val_csv = val_csv
        self.val_root = val_root
        self.test_csv = test_csv
        self.test_root = test_root
        self.patch_size = patch_size
        self.num_classes = num_classes
        self.down_ratio = down_ratio
    
    def build(self):
        """
        Construye los datasets de entrenamiento, validación y prueba.
        
        Retorna
        -------
        tuple
            (train_dataset, val_dataset, test_dataset)
        """
        # Dataset de entrenamiento con transformaciones de aumento
        train_dataset = CSVDataset(
            csv_file=self.train_csv,
            root_dir=self.train_root,
            albu_transforms=[
                A.VerticalFlip(p=0.5),
                A.HorizontalFlip(p=0.5),
                A.RandomRotate90(p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
                A.Blur(blur_limit=15, p=0.2),
                A.Normalize(p=1.0),
            ],
            end_transforms=[
                MultiTransformsWrapper([
                    FIDT(num_classes=self.num_classes, down_ratio=self.down_ratio),
                    PointsToMask(
                        radius=2,
                        num_classes=self.num_classes,
                        squeeze=True,
                        down_ratio=int(self.patch_size // 16),
                    ),
                ])
            ],
        )
        
        # Dataset de validación
        val_dataset = CSVDataset(
            csv_file=self.val_csv,
            root_dir=self.val_root,
            albu_transforms=[A.Normalize(p=1.0)],
            end_transforms=[DownSample(down_ratio=self.down_ratio, anno_type="point")],
        )
        
        # Dataset de prueba
        test_dataset = CSVDataset(
            csv_file=self.test_csv,
            root_dir=self.test_root,
            albu_transforms=[A.Normalize(p=1.0)],
            end_transforms=[DownSample(down_ratio=self.down_ratio, anno_type="point")],
        )
        
        # Retorna los tres datasets sin imprimir nada
        return train_dataset, val_dataset, test_dataset

# Crear instancia de la clase
builder = HerdNetDatasetsBuilder(
    train_csv='/content/data/train_patches.csv',
    train_root='/content/data/train_patches',
    val_csv='/content/data/val_patches/gt.csv',
    val_root='/content/data/val_patches',
    test_csv='/content/data/test.csv',
    test_root='/content/data/test'
)

# Construir los datasets
train_dataset, val_dataset, test_dataset = builder.build()

In [ ]:
# Dataloaders
from torch.utils.data import DataLoader

# Create dataloaders
train_dataloader = DataLoader(dataset = train_dataset, batch_size = 4, shuffle = True)
val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

# Define HerdNet for training

In [ ]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(pretrained=False, num_classes=num_classes, down_ratio=down_ratio).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
]

herdnet = LossWrapper(herdnet, losses=losses)

# URL (DLA): http://dl.yf.io/dla/models/imagenet/dla34-ba72cf86.pth

# Create the Trainer

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/drive/MyDrive/HerdNet/output'
mkdir(work_dir)

lr = 1e-4 
weight_decay = 1e-3
epochs = 100

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
)

evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=val_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=work_dir,
    header='validation'
)

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
)

# Start training

In [ ]:
trainer.start(warmup_iters=100, checkpoints='best', select='max', validate_on='f1_score')

# Test the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create output folder
test_dir = "/content/drive/MyDrive/HerdNet/test_p1"
mkdir(test_dir)

from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

num_classes = 7
down_ratio = 2

herdnet = HerdNet(pretrained=False, num_classes=num_classes, down_ratio=down_ratio).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
]

herdnet = LossWrapper(herdnet, losses=losses)

In [ ]:
# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path="/content/drive/MyDrive/HerdNet/fase_1/output/best_model.pth")

from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

lr = 1e-4
weight_decay = 1e-3
epochs = 100
num_classes = 7
patch_size = 512
down_ratio = 2

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
)

# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
)

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=test_evaluator,
    work_dir=test_dir
)

In [ ]:
# Start testing
import warnings
warnings.filterwarnings("ignore", message="Got processor for keypoints, but no transform to process it")

test_f1_score = test_evaluator.evaluate(returns='f1_score')

# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

In [ ]:
# Get the detections
detections = test_evaluator.results
detections

# Inferir sobre train (no patches)

In [ ]:
import torch
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import albumentations as A
import numpy as np
import os
from torch.utils.data import DataLoader
from animaloc.data.transforms import DownSample
from animaloc.models import HerdNet, LossWrapper
from animaloc.eval import HerdNetStitcher, HerdNetEvaluator
from animaloc.eval.metrics import PointsMetrics
from animaloc.datasets import CSVDataset
from animaloc.utils.useful_funcs import mkdir
from animaloc.vizual import draw_points, draw_text

def infer_herdnet(
    model_path,
    images_dir,
    output_dir,
    patch_size=512,
    overlap=160,
    down_ratio=2,
    device="cuda",
    up=True,
    reduction="mean",
    class_map=None,
    draw_results=True,
):
    """
    Run inference with HerdNet model and export detections and visualizations.
    
    Replicates and extends the original infer.py by Alexandre Delplanque.
    """
    mkdir(output_dir)
    
    print(f"[INFO] Loading model from: {model_path}")
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    
    # 1. Construcción del modelo
    num_classes = 7  # 6 especies + fondo
    model = HerdNet(num_classes=num_classes, down_ratio=down_ratio, pretrained=False)
    model = LossWrapper(model, [])
    
    checkpoint = torch.load(model_path, map_location=device)
    if "model_state_dict" in checkpoint:
        model.load_state_dict(checkpoint["model_state_dict"])
    else:
        model.load_state_dict(checkpoint)
    
    model.to(device)
    model.eval()
    print("[INFO] Model loaded successfully.")
    
    # 2. Crear DataLoader con imágenes completas
    img_names = [i for i in os.listdir(images_dir) if i.lower().endswith(('.jpg', '.jpeg'))]
    df = pd.DataFrame({'images': img_names, 'x': [0]*len(img_names), 'y': [0]*len(img_names), 'labels': [1]*len(img_names)})
    
    albu_transforms = [A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))]
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    
    dataset = CSVDataset(
        csv_file=df,
        root_dir=images_dir,
        albu_transforms=albu_transforms,
        end_transforms=end_transforms
    )
    
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
    
    # 3. Configurar stitcher y evaluator
    stitcher = HerdNetStitcher(
        model=model,
        size=(patch_size, patch_size),
        overlap=overlap,
        down_ratio=down_ratio,
        up=up,
        reduction=reduction,
        device_name=device,
    )
    
    metrics = PointsMetrics(5, num_classes=num_classes)
    
    evaluator = HerdNetEvaluator(
        model=model,
        dataloader=dataloader,
        metrics=metrics,
        lmds_kwargs=dict(kernel_size=(3,3), adapt_ts=0.2, neg_ts=0.1),
        device_name=device,
        print_freq=10,
        stitcher=stitcher,
        work_dir=output_dir,
        header='[INFERENCE]'
    )
    
    # 4. Ejecutar inferencia
    print("[INFO] Starting inference ...")
    evaluator.evaluate(wandb_flag=False, viz=False, log_meters=False)
    
    detections = evaluator.detections
    detections.dropna(inplace=True)
    
    # 5. Asignar nombres de especies
    if class_map is None:
        class_map = {
            1: 'buffalo', 2: 'elephant', 3: 'kob',
            4: 'topi', 5: 'warthog', 6: 'waterbuck'
        }
    
    detections['species'] = detections['labels'].map(class_map)
    
    # 6. Guardar CSV
    csv_path = Path(output_dir) / "detections.csv"
    detections.to_csv(csv_path, index=False)
    print(f"[INFO] Saved detections to: {csv_path}")
    
    # 7. Exportar visualizaciones (opcional)
    if draw_results and not detections.empty:
        print("[INFO] Exporting plots and thumbnails ...")
        dest_plots = Path(output_dir) / "plots"
        dest_thumbs = Path(output_dir) / "thumbnails"
        mkdir(dest_plots)
        mkdir(dest_thumbs)
        
        for img_name in tqdm(detections['images'].unique(), desc="Drawing"):
            img_path = os.path.join(images_dir, img_name)
            img = Image.open(img_path).convert("RGB")
            img_cpy = img.copy()
            
            pts = list(detections[detections['images'] == img_name][['y', 'x']].to_records(index=False))
            pts = [(y, x) for y, x in pts]
            
            output = draw_points(img, pts, color='red', size=10)
            output.save(dest_plots / img_name, quality=95)
            
            sp_score = list(detections[detections['images'] == img_name][['species','scores']].to_records(index=False))
            
            for i, ((y, x), (sp, score)) in enumerate(zip(pts, sp_score)):
                off = 128
                coords = (x - off, y - off, x + off, y + off)
                thumbnail = img_cpy.crop(coords)
                score = round(score * 100, 0)
                thumbnail = draw_text(thumbnail, f"{sp} | {score}%", position=(10,5), font_size=20)
                thumbnail.save(dest_thumbs / f"{img_name[:-4]}_{i}.JPG")
    
    # 8. Resumen final
    print("\n[INFO] Detection summary per species:")
    if not detections.empty:
        for sp, count in detections['species'].value_counts().items():
            print(f"  - {sp}: {count}")
        print(f"  Total detections: {len(detections)}")
    else:
        print("  No detections found.")
    
    return detections

In [ ]:
from datetime import datetime
import os

MODEL_PATH = "/content/drive/MyDrive/HerdNet/fase_1/best_model.pth"
IMAGES_DIR = "/content/data/train"
OUTPUT_DIR = f"/content/drive/MyDrive/HerdNet/fase_1/infer_train_{datetime.now().strftime('%Y%m%d')}"

detecciones = infer_herdnet(
    model_path=MODEL_PATH,
    images_dir=IMAGES_DIR,
    output_dir=OUTPUT_DIR,
    patch_size=512,
    overlap=160,
    down_ratio=2,
    device="cuda",
    draw_results=True
)

# Generar HNP

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

class HardNegativePatchMiner:
    """
    Clase para realizar minería de Hard Negative Patches (HNP) a partir de las detecciones
    y el conjunto de ground truth.
    
    Identifica falsos positivos comparando las predicciones del modelo con los puntos reales
    de cada imagen, considerando un radio de tolerancia. Las detecciones que no coinciden
    con ningún punto real dentro de dicho radio se consideran candidatos HNP.
    
    Parámetros
    ----------
    detections_path : str
        Ruta al archivo CSV con las detecciones del modelo.
    groundtruth_path : str
        Ruta al archivo CSV con las anotaciones reales (ground truth).
    output_path : str
        Ruta donde se guardará el archivo CSV con los candidatos HNP.
    radius : int, opcional
        Radio máximo de tolerancia para considerar una detección como válida. Por defecto es 20.
    score_threshold : float, opcional
        Umbral mínimo de confianza para conservar detecciones. Por defecto es 0.95.
    
    Retorna
    -------
    pd.DataFrame
        DataFrame con los registros de Hard Negative Patches (HNP).
    """
    
    def __init__(
        self,
        detections_path: str,
        groundtruth_path: str,
        output_path: str,
        radius: int = 20,
        score_threshold: float = 0.95,
    ):
        # Se guardan los parámetros de entrada
        self.detections_path = detections_path
        self.groundtruth_path = groundtruth_path
        self.output_path = output_path
        self.radius = radius
        self.score_threshold = score_threshold
    
    def mine(self):
        """
        Ejecuta el proceso de minería de Hard Negative Patches (HNP).
        
        Retorna
        -------
        pd.DataFrame
            DataFrame con los registros de HNP encontrados.
        """
        # Cargar detecciones y ground truth
        det = pd.read_csv(self.detections_path)
        gt = pd.read_csv(self.groundtruth_path)
        
        # Asegurar consistencia en los nombres de las imágenes
        det["images"] = det["images"].astype(str)
        gt["images"] = gt["images"].astype(str)
        
        # Filtrar detecciones por umbral de confianza
        det = det[det["scores"] >= self.score_threshold].reset_index(drop=True)
        
        # Lista donde se acumulan los registros de falsos positivos
        hnp_records = []
        
        # Evaluar imagen por imagen
        for img_name in tqdm(det["images"].unique(), desc="Mining HNPs"):
            preds = det[det["images"] == img_name]
            gts = gt[gt["images"] == img_name]
            
            # Si no existe ground truth, todas las predicciones son falsos positivos
            if gts.empty:
                for _, row in preds.iterrows():
                    hnp_records.append(row)
                continue
            
            # Coordenadas reales
            gt_points = gts[["x", "y"]].to_numpy()
            
            # Comparar cada predicción
            for _, row in preds.iterrows():
                px, py = row["x"], row["y"]
                distances = np.sqrt(((gt_points[:, 0] - px) ** 2) + ((gt_points[:, 1] - py) ** 2))
                
                # Si todas las distancias son mayores al radio, es falso positivo
                if np.all(distances > self.radius):
                    hnp_records.append(row)
        
        # Crear DataFrame de resultados
        hnp_df = pd.DataFrame(hnp_records)
        
        # Crear carpeta destino
        Path(self.output_path).parent.mkdir(parents=True, exist_ok=True)
        
        # Guardar resultados
        hnp_df.to_csv(self.output_path, index=False)
        
        # Retornar el DataFrame
        return hnp_df

class HardNegativePatchMinerV2:
    """
    Versión mejorada del minero de Hard Negative Patches.
    
    Mejoras:
    - Filtrado por percentiles de confianza en lugar de umbral fijo
    - Balanceo por clase para evitar sesgos
    - Filtrado de duplicados cercanos
    - Análisis de distribución espacial
    
    Parámetros
    ----------
    detections_path : str
        Ruta al archivo CSV con las detecciones del modelo.
    groundtruth_path : str
        Ruta al archivo CSV con las anotaciones reales (ground truth).
    output_path : str
        Ruta donde se guardará el archivo CSV con los candidatos HNP.
    radius : int, opcional
        Radio máximo de tolerancia para considerar una detección como válida. Por defecto es 20.
    score_percentile : float, opcional
        Percentil de confianza para filtrar detecciones. Por defecto es 0.7 (top 30%).
    max_samples_per_image : int, opcional
        Máximo número de HNPs por imagen. Por defecto es 10.
    min_distance_between_hnp : int, opcional
        Distancia mínima entre HNPs para evitar redundancia. Por defecto es 100.
    
    Retorna
    -------
    pd.DataFrame
        DataFrame con los registros de Hard Negative Patches (HNP).
    """
    
    def __init__(
        self,
        detections_path: str,
        groundtruth_path: str,
        output_path: str,
        radius: int = 20,
        score_percentile: float = 0.7,
        max_samples_per_image: int = 10,
        min_distance_between_hnp: int = 100,
    ):
        self.detections_path = detections_path
        self.groundtruth_path = groundtruth_path
        self.output_path = output_path
        self.radius = radius
        self.score_percentile = score_percentile
        self.max_samples_per_image = max_samples_per_image
        self.min_distance_between_hnp = min_distance_between_hnp
    
    def _is_far_from_existing(self, x, y, existing_points, min_dist):
        """Verifica si un punto está suficientemente lejos de los existentes."""
        if not existing_points:
            return True
        existing_array = np.array(existing_points)
        distances = np.sqrt((existing_array[:, 0] - x)**2 + (existing_array[:, 1] - y)**2)
        return np.all(distances > min_dist)
    
    def mine(self):
        """
        Ejecuta el minado mejorado de HNPs.
        
        Retorna
        -------
        pd.DataFrame
            DataFrame con los registros de HNP encontrados.
        """
        # Cargar datos
        det = pd.read_csv(self.detections_path)
        gt = pd.read_csv(self.groundtruth_path)
        
        det["images"] = det["images"].astype(str)
        gt["images"] = gt["images"].astype(str)
        
        # Calcular umbral dinámico basado en percentiles
        score_threshold = det["scores"].quantile(self.score_percentile)
        print(f"[INFO] Umbral de confianza calculado (percentil {self.score_percentile*100}%): {score_threshold:.3f}")
        
        # Filtrar por umbral
        det = det[det["scores"] >= score_threshold].reset_index(drop=True)
        
        hnp_records = []
        stats = {"total_candidates": 0, "filtered_by_distance": 0, "filtered_by_max_samples": 0}
        
        for img_name in tqdm(det["images"].unique(), desc="Mining HNPs v2"):
            preds = det[det["images"] == img_name]
            gts = gt[gt["images"] == img_name]
            
            image_hnps = []
            existing_hnp_coords = []
            
            # Si no hay GT, todas son FP
            if gts.empty:
                for _, row in preds.iterrows():
                    if len(image_hnps) >= self.max_samples_per_image:
                        stats["filtered_by_max_samples"] += 1
                        continue
                    
                    x, y = row["x"], row["y"]
                    if self._is_far_from_existing(x, y, existing_hnp_coords, self.min_distance_between_hnp):
                        image_hnps.append(row)
                        existing_hnp_coords.append((x, y))
                        stats["total_candidates"] += 1
                    else:
                        stats["filtered_by_distance"] += 1
                continue
            
            gt_points = gts[["x", "y"]].to_numpy()
            
            # Ordenar predicciones por confianza (descendente)
            preds_sorted = preds.sort_values("scores", ascending=False)
            
            for _, row in preds_sorted.iterrows():
                if len(image_hnps) >= self.max_samples_per_image:
                    stats["filtered_by_max_samples"] += 1
                    continue
                
                px, py = row["x"], row["y"]
                distances = np.sqrt((gt_points[:, 0] - px)**2 + (gt_points[:, 1] - py)**2)
                
                # Si es falso positivo
                if np.all(distances > self.radius):
                    # Verificar distancia con otros HNPs
                    if self._is_far_from_existing(px, py, existing_hnp_coords, self.min_distance_between_hnp):
                        image_hnps.append(row)
                        existing_hnp_coords.append((px, py))
                        stats["total_candidates"] += 1
                    else:
                        stats["filtered_by_distance"] += 1
            
            hnp_records.extend(image_hnps)
        
        hnp_df = pd.DataFrame(hnp_records)
        
        # Estadísticas
        print(f"\n[INFO] Estadísticas de minado:")
        print(f"  - Candidatos totales: {stats['total_candidates']}")
        print(f"  - Filtrados por distancia: {stats['filtered_by_distance']}")
        print(f"  - Filtrados por máximo/imagen: {stats['filtered_by_max_samples']}")
        print(f"  - HNPs finales: {len(hnp_df)}")
        
        # Guardar
        Path(self.output_path).parent.mkdir(parents=True, exist_ok=True)
        hnp_df.to_csv(self.output_path, index=False)
        
        return hnp_df

# Usar la versión mejorada
miner = HardNegativePatchMinerV2(
    detections_path="/content/drive/MyDrive/HerdNet/fase_1/infer_train_20251101/detections.csv",
    groundtruth_path="/content/data/train.csv",
    output_path="/content/data/hnp_candidates.csv",
    radius=20,
    score_percentile=0.6,
    max_samples_per_image=15,
    min_distance_between_hnp=80
)

hnp_df = miner.mine()

In [ ]:
import os
from pathlib import Path
import pandas as pd
from PIL import Image
from tqdm import tqdm

class HNPPatchesExtractor:
    """
    Clase para extraer parches de Hard Negative Patches (HNP) a partir de un
    archivo CSV con coordenadas de detecciones falsas positivas.
    
    Crea imágenes recortadas centradas en las coordenadas de cada detección,
    guardándolas en una carpeta destino junto con un archivo CSV que registra
    los metadatos de cada parche generado.
    
    Parámetros
    ----------
    hnp_csv : str
        Ruta al archivo CSV con las detecciones HNP.
    images_root : str
        Carpeta raíz donde se encuentran las imágenes originales.
    output_dir : str
        Carpeta destino donde se guardarán los parches generados.
    patch_size : int, opcional
        Tamaño del parche cuadrado a recortar. Por defecto es 512 píxeles.
    
    Retorna
    -------
    pd.DataFrame
        DataFrame con los registros de los parches generados, incluyendo
        nombre del archivo, coordenadas (x, y) y etiqueta (labels = 0).
    """
    
    def __init__(
        self,
        hnp_csv: str,
        images_root: str,
        output_dir: str,
        patch_size: int = 512,
    ):
        # Se guardan los parámetros como atributos
        self.hnp_csv = hnp_csv
        self.images_root = images_root
        self.output_dir = output_dir
        self.patch_size = patch_size
    
    def extract(self):
        """
        Ejecuta la extracción de parches HNP y genera el archivo CSV de salida.
        
        Retorna
        -------
        pd.DataFrame
            DataFrame con los metadatos de los parches generados.
        """
        # Crear carpeta de salida si no existe
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)
        
        # Cargar el archivo CSV con las detecciones negativas
        hnp_df = pd.read_csv(self.hnp_csv)
        
        # Lista para almacenar los registros de salida
        records = []
        
        half = self.patch_size // 2
        
        # Iterar sobre cada detección y extraer parches
        for idx, row in tqdm(hnp_df.iterrows(), total=len(hnp_df), desc="Extracting patches"):
            img_name = row["images"]
            img_path = os.path.join(self.images_root, img_name)
            
            # Omitir si la imagen no existe
            if not os.path.exists(img_path):
                continue
            
            try:
                # Abrir imagen y obtener dimensiones
                img = Image.open(img_path).convert("RGB")
                w, h = img.size
                
                # Coordenadas del centro de la detección
                x, y = int(row["x"]), int(row["y"])
                
                # Calcular límites del parche (centrado)
                left = max(0, x - half)
                upper = max(0, y - half)
                right = min(w, x + half)
                lower = min(h, y + half)
                
                # Extraer y guardar el parche
                patch = img.crop((left, upper, right, lower))
                patch_name = f"{Path(img_name).stem}_hnp_{idx:05d}.JPG"
                patch.save(os.path.join(self.output_dir, patch_name), quality=95)
                
                # Registrar metadatos
                records.append({"images": patch_name, "x": x, "y": y, "labels": 0})
            
            except Exception:
                # Se ignoran errores de lectura o recorte
                continue
        
        # Guardar CSV final con los registros
        hnp_patches_csv = os.path.join(self.output_dir, "hnp_patches.csv")
        pd.DataFrame(records).to_csv(hnp_patches_csv, index=False)
        
        # Retornar los registros
        return pd.DataFrame(records)

extractor = HNPPatchesExtractor(
    hnp_csv="/content/data/hnp_candidates.csv",
    images_root="/content/data/train",
    output_dir="/content/data/hnp_patches",
    patch_size=512
)

hnp_patches_df = extractor.extract()

In [ ]:
import shutil
from pathlib import Path

class TrainPatchesMerger:
    """
    Clase para combinar los parches originales de entrenamiento con los parches
    negativos (Hard Negative Patches, HNP) en una única carpeta consolidada.
    
    Copia todos los archivos .JPG desde las carpetas de entrenamiento y HNP hacia
    una carpeta final de parches fusionados.
    
    Parámetros
    ----------
    train_patches_dir : str o Path
        Carpeta que contiene los parches de entrenamiento originales.
    hnp_patches_dir : str o Path
        Carpeta que contiene los parches negativos (HNP).
    merged_dir : str o Path
        Carpeta donde se guardarán los parches fusionados.
    
    Retorna
    -------
    tuple
        Una tupla con:
        - total_patches (int): número total de parches copiados.
        - merged_dir (Path): ruta de la carpeta consolidada.
    """
    
    def __init__(self, train_patches_dir, hnp_patches_dir, merged_dir):
        # Se convierten las rutas a objetos Path para operaciones consistentes
        self.train_patches_dir = Path(train_patches_dir)
        self.hnp_patches_dir = Path(hnp_patches_dir)
        self.merged_dir = Path(merged_dir)
    
    def merge(self):
        """
        Ejecuta la fusión de parches de entrenamiento y negativos.
        
        Retorna
        -------
        tuple
            (total_patches, merged_dir)
        """
        # Crear la carpeta de salida si no existe
        self.merged_dir.mkdir(parents=True, exist_ok=True)
        
        # Copiar los parches de entrenamiento originales
        for img in self.train_patches_dir.glob("*.JPG"):
            shutil.copy(img, self.merged_dir / img.name)
        
        # Copiar los parches negativos
        for img in self.hnp_patches_dir.glob("*.JPG"):
            shutil.copy(img, self.merged_dir / img.name)
        
        # Contar el total de archivos copiados
        total_patches = len(list(self.merged_dir.glob("*.JPG")))
        
        # Retornar resultados
        return total_patches, self.merged_dir

merger = TrainPatchesMerger(
    train_patches_dir="/content/data/train_patches",
    hnp_patches_dir="/content/data/hnp_patches",
    merged_dir="/content/data/merged_train_patches"
)

total_patches, merged_path = merger.merge()

class BalancedPatchSampler:
    """
    Balancea el dataset considerando clases positivas y negativas.
    
    Parámetros
    ----------
    csv_path : str
        Ruta al archivo CSV con todos los parches.
    target_ratio : float, opcional
        Proporción deseada de HNPs respecto al total. Por defecto es 0.3.
    
    Retorna
    -------
    pd.DataFrame
        DataFrame balanceado con parches positivos y negativos.
    """
    
    def __init__(self, csv_path, target_ratio=0.3):
        self.csv_path = csv_path
        self.target_ratio = target_ratio
    
    def balance(self, output_path):
        """
        Crea un CSV balanceado.
        
        Parámetros
        ----------
        output_path : str
            Ruta donde se guardará el CSV balanceado.
        
        Retorna
        -------
        pd.DataFrame
            DataFrame balanceado.
        """
        df = pd.read_csv(self.csv_path)
        
        # Separar positivos y negativos
        positives = df[df['labels'] != 0]
        negatives = df[df['labels'] == 0]
        
        print(f"[INFO] Parches originales:")
        print(f"  - Positivos: {len(positives)}")
        print(f"  - Negativos (HNP): {len(negatives)}")
        
        # Calcular cantidad objetivo de negativos
        n_positives = len(positives)
        n_negatives_target = int(n_positives * self.target_ratio / (1 - self.target_ratio))
        
        # Subsamplear negativos si hay demasiados
        if len(negatives) > n_negatives_target:
            negatives = negatives.sample(n=n_negatives_target, random_state=9292)
            print(f"[INFO] Negativos reducidos a: {len(negatives)}")
        
        # Combinar
        balanced_df = pd.concat([positives, negatives], ignore_index=True)
        balanced_df = balanced_df.sample(frac=1, random_state=9292).reset_index(drop=True)
        
        print(f"[INFO] Dataset balanceado final: {len(balanced_df)} parches")
        print(f"  - Proporción HNP: {len(negatives)/len(balanced_df)*100:.1f}%")
        
        balanced_df.to_csv(output_path, index=False)
        return balanced_df

# Aplicar balanceo ANTES de crear el dataset de Fase 2
balancer = BalancedPatchSampler(
    csv_path="/content/data/train_patches.csv",
    target_ratio=0.25
)

balanced_csv = balancer.balance("/content/data/train_patches_balanced.csv")

# Segunda fase

In [ ]:
# CONFIGURACIÓN GENERAL - FASE 2 (HARD NEGATIVE PATCHES)

import os
import torch
from torch import Tensor
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import albumentations as A
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss
from animaloc.datasets import FolderDataset, CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, FIDT, PointsToMask, DownSample
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

# PARÁMETROS
PATCH_SIZE = 512
DOWN_RATIO = 2
NUM_CLASSES = 7
WORK_DIR = "/content/drive/MyDrive/HerdNet/fase_2"
mkdir(WORK_DIR)

# DATASETS
from albumentations import PadIfNeeded

train_dataset = FolderDataset(
    csv_file="/content/data/train_patches_balanced.csv",
    root_dir="/content/data/merged_train_patches",
    albu_transforms=[
        A.PadIfNeeded(min_height=512, min_width=512, border_mode=0, p=1.0),
        
        # Transformaciones geométricas (mantener altas)
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.3),
        
        # Transformaciones de color (moderadas)
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.2),
        
        # Ruido y blur (moderados)
        A.OneOf([
            A.Blur(blur_limit=7, p=1.0),
            A.GaussNoise(var_limit=(10.0, 30.0), p=1.0),
            A.MedianBlur(blur_limit=5, p=1.0),
        ], p=0.2),
        
        # Normalización (siempre)
        A.Normalize(p=1.0)
    ],
    end_transforms=[MultiTransformsWrapper([
        FIDT(num_classes=NUM_CLASSES, down_ratio=DOWN_RATIO),
        PointsToMask(radius=2, num_classes=NUM_CLASSES, squeeze=True, down_ratio=int(PATCH_SIZE // 16))
    ])]
)

print(f"[INFO] Dataset de entrenamiento cargado: {len(train_dataset)} parches (positivos + HNPs)")

# Validación y test (idénticos a Fase 1)
val_dataset = CSVDataset(
    csv_file="/content/data/val_patches/gt.csv",
    root_dir="/content/data/val_patches",
    albu_transforms=[A.Normalize(p=1.0)],
    end_transforms=[DownSample(down_ratio=DOWN_RATIO, anno_type="point")]
)

test_dataset = CSVDataset(
    csv_file="/content/data/test.csv",
    root_dir="/content/data/test",
    albu_transforms=[A.Normalize(p=1.0)],
    end_transforms=[DownSample(down_ratio=DOWN_RATIO, anno_type="point")]
)


# DATALOADERS
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(pretrained=False, num_classes=NUM_CLASSES, down_ratio=DOWN_RATIO).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
]

herdnet = LossWrapper(herdnet, losses=losses)

# Load trained parameters
from animaloc.models import load_model

# Se interrumpió al 32 epoch. Se reanuda
herdnet = load_model(herdnet, pth_path="/content/drive/MyDrive/HerdNet/fase_2/other/best_model.pth")

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/drive/MyDrive/HerdNet/fase_2/output'
mkdir(work_dir)

class EarlyStoppingCallback:
    """
    Implementa early stopping basado en una métrica de validación.
    
    Parámetros
    ----------
    patience : int, opcional
        Número de épocas sin mejora antes de detener. Por defecto es 10.
    min_delta : float, opcional
        Mejora mínima requerida para considerar progreso. Por defecto es 0.001.
    mode : str, opcional
        'max' para maximizar la métrica, 'min' para minimizar. Por defecto es 'max'.
    """
    
    def __init__(self, patience=10, min_delta=0.001, mode='max'):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
    
    def __call__(self, current_score):
        """
        Evalúa si se debe detener el entrenamiento.
        
        Parámetros
        ----------
        current_score : float
            Valor actual de la métrica de validación.
        
        Retorna
        -------
        bool
            True si se debe detener el entrenamiento, False en caso contrario.
        """
        if self.best_score is None:
            self.best_score = current_score
            return False
        
        if self.mode == 'max':
            score_improved = current_score > (self.best_score + self.min_delta)
        else:
            score_improved = current_score < (self.best_score - self.min_delta)
        
        if score_improved:
            self.best_score = current_score
            self.counter = 0
        else:
            self.counter += 1
            print(f"[EarlyStopping] No improvement for {self.counter}/{self.patience} epochs")
            
            if self.counter >= self.patience:
                self.early_stop = True
                print("[EarlyStopping] Stopping training!")
                return True
        
        return False

LR = 5e-5
WEIGHT_DECAY = 0.0005
EPOCHS = 50
OVERLAP = 160
RADIUS = 20
PATCH_SIZE = 512
PATIENCE = 15

optimizer = Adam(params=herdnet.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Scheduler para reducir LR cuando la métrica se estanque
scheduler = ReduceLROnPlateau(
    optimizer, 
    mode='max', 
    factor=0.5, 
    patience=5, 
    verbose=True,
    min_lr=1e-7
)

# Early stopping
early_stopping = EarlyStoppingCallback(patience=PATIENCE, min_delta=0.002, mode='max')

metrics = PointsMetrics(radius=RADIUS, num_classes=NUM_CLASSES)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(PATCH_SIZE,PATCH_SIZE),
    overlap=OVERLAP,
    down_ratio=DOWN_RATIO,
    reduction='mean'
)

evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=val_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=work_dir,
    header='validation'
)

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=EPOCHS,
    evaluator=evaluator,
    work_dir=work_dir
)

In [ ]:
# Start testing
import warnings
warnings.filterwarnings("ignore", message="Got processor for keypoints, but no transform to process it")

print("[INFO] Iniciando entrenamiento Fase 2 con scheduler y early stopping...")

# Entrenar por épocas manualmente para integrar scheduler
for epoch in range(EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"{'='*60}")
    
    # Entrenar una época
    trainer.train_epoch()
    
    # Validar
    val_metrics = evaluator.evaluate(returns='f1_score', wandb_flag=False)
    current_f1 = val_metrics if isinstance(val_metrics, float) else val_metrics.get('f1_score', 0)
    
    print(f"\n[Validation] F1-Score: {current_f1:.4f}")
    
    # Actualizar scheduler
    scheduler.step(current_f1)
    
    # Verificar early stopping
    if early_stopping(current_f1):
        print(f"\n[INFO] Early stopping activado en época {epoch+1}")
        break
    
    # Guardar mejor modelo
    if current_f1 > trainer.best_score:
        trainer.best_score = current_f1
        trainer.save_checkpoint('best')
        print(f"[INFO] Nuevo mejor modelo guardado (F1: {current_f1:.4f})")

print("\n[INFO] Entrenamiento completado")

In [ ]:
!cp content/data content/drive/MyDrive/HerdNet/dataset -r

In [ ]:
# Create output folder
test_dir = "/content/drive/MyDrive/HerdNet/test_p2"
mkdir(test_dir)

In [ ]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

num_classes = 7
patch_size = 512
down_ratio = 2

herdnet = HerdNet(pretrained=False, num_classes=num_classes, down_ratio=down_ratio).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
]

herdnet = LossWrapper(herdnet, losses=losses)

In [ ]:
# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path="/content/drive/MyDrive/HerdNet/fase_2/output/best_model.pth")

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

lr = 1e-4
weight_decay = 1e-3
epochs = 100
num_classes = 7

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
)

def evaluate_with_class_metrics(evaluator, dataloader, work_dir):
    """
    Evalúa el modelo y calcula métricas por clase.
    
    Parámetros
    ----------
    evaluator : HerdNetEvaluator
        Evaluador configurado del modelo.
    dataloader : DataLoader
        DataLoader con los datos de evaluación.
    work_dir : str
        Directorio de trabajo para guardar resultados.
    
    Retorna
    -------
    dict
        Diccionario con todas las métricas calculadas.
    """
    results = evaluator.evaluate(returns='all', wandb_flag=False)
    
    # Obtener detecciones
    detections = evaluator.results
    
    # Mapeo de clases
    class_map = {
        1: 'buffalo', 2: 'elephant', 3: 'kob',
        4: 'topi', 5: 'warthog', 6: 'waterbuck'
    }
    
    print("\n" + "="*60)
    print("MÉTRICAS POR CLASE")
    print("="*60)
    
    # Métricas globales
    global_f1 = results.get('f1_score', 0)
    global_precision = results.get('precision', 0)
    global_recall = results.get('recall', 0)
    
    print(f"\nGlobal:")
    print(f"  F1-Score:  {global_f1*100:.2f}%")
    print(f"  Precision: {global_precision*100:.2f}%")
    print(f"  Recall:    {global_recall*100:.2f}%")
    
    # Por clase (si están disponibles)
    if 'labels' in detections.columns and len(detections) > 0:
        print(f"\nDetecciones por especie:")
        for label, species in class_map.items():
            count = len(detections[detections['labels'] == label])
            print(f"  {species:12s}: {count:4d}")
    
    print("="*60 + "\n")
    
    return results

# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
)

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=test_evaluator,
    work_dir=test_dir
)

In [ ]:
# Start testing
import warnings
warnings.filterwarnings("ignore", message="Got processor for keypoints, but no transform to process it")

final_results = evaluate_with_class_metrics(test_evaluator, test_dataloader, test_dir)

test_f1_score = final_results.get('f1_score', 0)

# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

In [ ]:
# Get the detections
detections = test_evaluator.results
detections

In [ ]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

num_classes = 7
patch_size = 512
down_ratio = 2

herdnet = HerdNet(pretrained=False, num_classes=num_classes, down_ratio=down_ratio).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
]

herdnet = LossWrapper(herdnet, losses=losses)

# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path="/content/drive/MyDrive/HerdNet/fase_2/backup/best_model.pth")

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

lr = 1e-4
weight_decay = 1e-3
epochs = 100
num_classes = 7

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
)

# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
)

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=test_evaluator,
    work_dir=test_dir
)

In [ ]:
# Start testing
import warnings
warnings.filterwarnings("ignore", message="Got processor for keypoints, but no transform to process it")

final_results = evaluate_with_class_metrics(test_evaluator, test_dataloader, test_dir)

test_f1_score = final_results.get('f1_score', 0)

# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

In [ ]:
# Get the detections
detections = test_evaluator.results
detections

# Mejora implementada

In [ ]:
# Inferir sobre train (no patches)

import torch
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import albumentations as A
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from animaloc.models import HerdNet
from animaloc.eval import HerdNetStitcher
from animaloc.utils.useful_funcs import mkdir
from animaloc.vizual import draw_points, draw_text


class SimpleImageDataset(Dataset):
    """
    Dataset simple para inferencia sin anotaciones.
    
    Parámetros
    ----------
    images_dir : str
        Directorio con las imágenes.
    transform : callable, opcional
        Transformaciones de Albumentations a aplicar.
    """
    
    def __init__(self, images_dir, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.image_names = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg'))]
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.images_dir, img_name)
        
        image = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        
        # Convertir a tensor
        image = torch.from_numpy(image).permute(2, 0, 1).float()
        
        return {'image': image, 'name': img_name}


def infer_herdnet(
    model_path,
    images_dir,
    output_dir,
    patch_size=512,
    overlap=160,
    down_ratio=2,
    device="cuda",
    up=True,
    reduction="mean",
    class_map=None,
    draw_results=True,
):
    """
    Run inference with HerdNet model and export detections and visualizations.
    
    Versión mejorada sin dependencias innecesarias de CSVDataset y Evaluator.
    
    Parámetros
    ----------
    model_path : str
        Ruta al modelo entrenado (.pth).
    images_dir : str
        Directorio con las imágenes para inferencia.
    output_dir : str
        Directorio de salida para resultados.
    patch_size : int, opcional
        Tamaño de parche para stitching. Por defecto 512.
    overlap : int, opcional
        Solapamiento entre parches. Por defecto 160.
    down_ratio : int, opcional
        Factor de reducción del modelo. Por defecto 2.
    device : str, opcional
        Dispositivo de cómputo. Por defecto "cuda".
    up : bool, opcional
        Si se upsamplea la salida. Por defecto True.
    reduction : str, opcional
        Método de reducción en solapamientos. Por defecto "mean".
    class_map : dict, opcional
        Mapeo de labels a nombres de especies.
    draw_results : bool, opcional
        Si se generan visualizaciones. Por defecto True.
    
    Retorna
    -------
    pd.DataFrame
        DataFrame con todas las detecciones.
    """
    mkdir(output_dir)
    
    print(f"[INFO] Loading model from: {model_path}")
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    
    # 1. Construcción del modelo
    num_classes = 7
    model = HerdNet(num_classes=num_classes, down_ratio=down_ratio, pretrained=False)
    
    checkpoint = torch.load(model_path, map_location=device)
    
    if "model_state_dict" in checkpoint:
        model.load_state_dict(checkpoint["model_state_dict"])
    else:
        model.load_state_dict(checkpoint)
    
    model.to(device)
    model.eval()
    print("[INFO] Model loaded successfully.")
    
    # 2. Crear Dataset y DataLoader
    transform = A.Compose([
        A.Normalize(p=1.0)
    ])
    
    dataset = SimpleImageDataset(images_dir=images_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)
    
    print(f"[INFO] Found {len(dataset)} images for inference")
    
    # 3. Configurar stitcher
    stitcher = HerdNetStitcher(
        model=model,
        size=(patch_size, patch_size),
        overlap=overlap,
        down_ratio=down_ratio,
        up=up,
        reduction=reduction,
        device_name=device,
    )
    
    # 4. Ejecutar inferencia
    print("[INFO] Starting inference...")
    all_detections = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Processing images"):
            images = batch['image'].to(device)
            img_names = batch['name']
            
            # Obtener predicciones con stitcher
            outputs = stitcher(images)
            
            # Procesar salidas
            heatmaps = outputs[0]
            class_maps = outputs[1]
            
            for i, img_name in enumerate(img_names):
                heatmap = heatmaps[i].cpu().numpy()
                class_map_pred = class_maps[i].cpu().numpy()
                
                # Detección de picos locales
                from scipy.ndimage import maximum_filter
                
                # Aplicar filtro de máximos locales
                kernel_size = 3
                local_max = maximum_filter(heatmap, size=kernel_size)
                
                # Umbrales adaptativos
                adapt_threshold = 0.2
                neg_threshold = 0.1
                
                # Máscara de detecciones
                detection_mask = (heatmap == local_max) & (heatmap > adapt_threshold)
                
                # Extraer coordenadas y scores
                coords = np.column_stack(np.where(detection_mask))
                
                for coord in coords:
                    y_pred, x_pred = coord
                    
                    # Escalar coordenadas al tamaño original
                    x_scaled = int(x_pred * down_ratio)
                    y_scaled = int(y_pred * down_ratio)
                    
                    score = float(heatmap[y_pred, x_pred])
                    
                    # Obtener clase predicha
                    if class_map_pred.ndim == 3:
                        label = int(np.argmax(class_map_pred[:, y_pred, x_pred]))
                    else:
                        label = int(class_map_pred[y_pred, x_pred])
                    
                    # Filtrar clase 0 (fondo)
                    if label > 0:
                        all_detections.append({
                            'images': img_name,
                            'x': x_scaled,
                            'y': y_scaled,
                            'labels': label,
                            'scores': score
                        })
    
    # 5. Crear DataFrame de detecciones
    detections = pd.DataFrame(all_detections)
    
    if detections.empty:
        print("[WARNING] No detections found!")
        detections = pd.DataFrame(columns=['images', 'x', 'y', 'labels', 'scores', 'species'])
    
    # 6. Asignar nombres de especies
    if class_map is None:
        class_map = {
            1: 'buffalo', 2: 'elephant', 3: 'kob',
            4: 'topi', 5: 'warthog', 6: 'waterbuck'
        }
    
    if not detections.empty:
        detections['species'] = detections['labels'].map(class_map)
    
    # 7. Guardar CSV
    csv_path = Path(output_dir) / "detections.csv"
    detections.to_csv(csv_path, index=False)
    print(f"[INFO] Saved detections to: {csv_path}")
    
    # 8. Exportar visualizaciones (opcional)
    if draw_results and not detections.empty:
        print("[INFO] Exporting plots and thumbnails...")
        dest_plots = Path(output_dir) / "plots"
        dest_thumbs = Path(output_dir) / "thumbnails"
        mkdir(dest_plots)
        mkdir(dest_thumbs)
        
        for img_name in tqdm(detections['images'].unique(), desc="Drawing"):
            img_path = os.path.join(images_dir, img_name)
            
            if not os.path.exists(img_path):
                continue
            
            img = Image.open(img_path).convert("RGB")
            img_cpy = img.copy()
            
            img_detections = detections[detections['images'] == img_name]
            pts = list(img_detections[['y', 'x']].to_records(index=False))
            pts = [(int(y), int(x)) for y, x in pts]
            
            output = draw_points(img, pts, color='red', size=10)
            output.save(dest_plots / img_name, quality=95)
            
            sp_score = list(img_detections[['species', 'scores']].to_records(index=False))
            
            for idx, ((y, x), (sp, score)) in enumerate(zip(pts, sp_score)):
                off = 128
                coords = (x - off, y - off, x + off, y + off)
                
                # Asegurar que las coordenadas estén dentro de los límites
                w, h = img_cpy.size
                coords = (
                    max(0, coords[0]),
                    max(0, coords[1]),
                    min(w, coords[2]),
                    min(h, coords[3])
                )
                
                thumbnail = img_cpy.crop(coords)
                score_pct = round(score * 100, 0)
                thumbnail = draw_text(thumbnail, f"{sp} | {score_pct}%", position=(10, 5), font_size=20)
                thumbnail.save(dest_thumbs / f"{img_name[:-4]}_{idx}.JPG")
    
    # 9. Resumen final
    print("\n[INFO] Detection summary per species:")
    if not detections.empty:
        for sp, count in detections['species'].value_counts().items():
            print(f"  - {sp}: {count}")
        print(f"  Total detections: {len(detections)}")
    else:
        print("  No detections found.")
    
    return detections


from datetime import datetime
import os

MODEL_PATH = "/content/drive/MyDrive/HerdNet/fase_1/best_model.pth"
IMAGES_DIR = "/content/data/train"
OUTPUT_DIR = f"/content/drive/MyDrive/HerdNet/fase_1/infer_train_{datetime.now().strftime('%Y%m%d')}"

detecciones = infer_herdnet(
    model_path=MODEL_PATH,
    images_dir=IMAGES_DIR,
    output_dir=OUTPUT_DIR,
    patch_size=512,
    overlap=160,
    down_ratio=2,
    device="cuda",
    draw_results=True
)

# ajustes de parámetros

1. Fase 1 - Entrenamiento inicial:

batch_size: 4 a 8 (aprovechar mejor GPU)
epochs: 100 a 80 (suficiente con early stopping)
warmup_iters: 100 a 200 (estabilización inicial)

2. HNP Mining (HardNegativePatchMinerV2):

score_percentile: 0.6 a 0.5 (capturar más FPs difíciles)
max_samples_per_image: 15 a 12 (evitar sobrerrepresentación)
min_distance_between_hnp: 80 a 100 (mayor diversidad espacial)

3. Balanceo (BalancedPatchSampler):

target_ratio: 0.25 a 0.20 (reducir HNPs al 20%, más conservador)

4. Fase 2 - Data Augmentation:

ShiftScaleRotate rotate_limit: 15 a 20 (mayor rotación)
HueSaturationValue sat_shift_limit: 20 a 15 (menos saturación)
batch_size: 16 a 12 (evitar OOM con augmentation pesado)

5. Fase 2 - Optimización:

LR: 5e-5 a 3e-5 (más conservador para fine-tuning)
EPOCHS: 50 a 40 (con early stopping es suficiente)
PATIENCE (early stopping): 15 a 12 (detener antes)
scheduler patience: 5 a 4 (reducir LR más rápido)

6. Inferencia:

overlap: 160 a 192 (mejor stitching, ~37.5%)
adapt_threshold: 0.2 a 0.3 (reducir FPs)
kernel_size (local_max): 3 a 5 (suprimir picos cercanos)

------ Others -----

- FocalLoss gamma: tunear gamma para controlar hard examples; probar {0.5, 1.0, 2.0}; empezar en 1.0 y bajar a 0.5 si recall cae mucho en clases
pequeñas.
- Combinar losses (lambda): variar lambdas entre detecciones y mapa de clases; probar lambda_ce ∈ {0.5,1.0,2.0} manteniendo focal λ=1 para priorizar 
robustez.


- Optimizadores alternativos: probar AdamW (mejor manejo weight decay) y RAdam para estabilidad;  y comparas. Si overfitting, AdamW suele ayudar.
- LR schedule: además de ReduceLROnPlateau, evaluar: CosineAnnealingWarmRestarts. ranges: initial LR 3e-5–1e-4 

- Memoria, batch y precisión numérica:
- Mixed precision (AMP): activar FP16 (apex o native torch.cuda.amp) para ganar batch sin OOM y acelerar.

... And more (jejeje).